# Soccer Tracking Pipeline

This notebook demonstrates the complete soccer tracking pipeline for processing MOT format datasets.

## 1. Setup and Imports

In [ ]:
import sys
from pathlib import Path
import os
import time
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display, Video, HTML

# Add src to path
sys.path.append('..')

from src.data import MOTDataLoader
from src.tracking import YOLOTracker, BotSortTracker
from src.evaluation import MOTEvaluator
from src.utils.visualization import plot_tracking_statistics
from src.utils.file_utils import ensure_dir
from config.paths import *

## 2. Data Setup

In [ ]:
# Check if we're in Colab and setup data paths
try:
    import google.colab
    IN_COLAB = True
    print("Running in Google Colab")
    
    # Use Colab paths
    dataset_path = LOCAL_DATASET_PATH
    results_dir = LOCAL_RESULTS_DIR
    
except ImportError:
    IN_COLAB = False
    print("Running locally")
    
    # Use local paths
    dataset_path = "../data/sample_dataset"
    results_dir = "../output/tracking_results"

print(f"Dataset path: {dataset_path}")
print(f"Results directory: {results_dir}")

# Create output directories
ensure_dir(results_dir)
ensure_dir("../output/videos")
ensure_dir("../output/plots")

## 3. Load Dataset

In [ ]:
# Load MOT dataset
if os.path.exists(dataset_path):
    print(f"Loading dataset from: {dataset_path}")
    data_loader = MOTDataLoader(dataset_path)
    
    # Get available sequences
    sequences = data_loader.get_sequence_list()
    print(f"Found {len(sequences)} sequences: {sequences}")
    
    # Display dataset information
    for seq in sequences[:5]:  # Show first 5 sequences
        info = data_loader.get_sequence_info(seq)
        print(f"\nSequence: {seq}")
        print(f"  Length: {info['length']} frames")
        print(f"  Resolution: {info['width']}x{info['height']}")
        print(f"  FPS: {info['fps']}")
        
        # Check if detection file exists
        detection_file = data_loader.detections_dir / f"{seq}.txt"
        if detection_file.exists():
            detections = data_loader.load_detections(str(detection_file))
            total_detections = sum(len(dets) for dets in detections.values())
            print(f"  Detections: {total_detections} total")
        else:
            print(f"  Detections: No detection file found")
            
else:
    print(f"Dataset not found at: {dataset_path}")
    print("Please run the setup notebook first to download/copy the dataset.")
    
    # Create a dummy dataset for demonstration
    print("\nCreating dummy dataset for demonstration...")
    sequences = ["demo_seq"]
    data_loader = None

## 4. Tracker Comparison

In [ ]:
# Initialize trackers for comparison
trackers = {}

# YOLO Tracker
print("Initializing YOLO tracker...")
try:
    trackers['YOLO'] = YOLOTracker(
        model_name='yolov8n.pt',
        confidence=0.3,
        device='auto'
    )
    print("✓ YOLO tracker ready")
except Exception as e:
    print(f"✗ YOLO tracker failed: {e}")

# BotSort Tracker
print("\nInitializing BotSort tracker...")
try:
    trackers['BotSort'] = BotSortTracker(
        device='auto' if 'cuda' in str(torch.cuda.is_available()) else 'cpu',
        with_reid=False
    )
    print("✓ BotSort tracker ready")
except Exception as e:
    print(f"✗ BotSort tracker failed: {e}")
    print("Note: BotSort requires 'boxmot' package. Install with: pip install boxmot")

print(f"\nInitialized {len(trackers)} trackers: {list(trackers.keys())}")

## 5. Process Sequences

In [ ]:
# Select sequence to process
if sequences and data_loader:
    test_sequence = sequences[0]  # Use first sequence
    print(f"Processing sequence: {test_sequence}")
    
    # Load sequence data
    detection_file = data_loader.detections_dir / f"{test_sequence}.txt"
    
    if detection_file.exists():
        detections_by_frame = data_loader.load_detections(str(detection_file))
        frames = data_loader.load_sequence_frames(test_sequence)
        
        print(f"Loaded {len(frames)} frames and {len(detections_by_frame)} detection frames")
        
        # Process with each tracker
        tracking_results = {}
        
        for tracker_name, tracker in trackers.items():
            print(f"\nProcessing with {tracker_name} tracker...")
            start_time = time.time()
            
            # Reset tracker
            tracker.reset()
            
            if tracker_name == 'YOLO':
                # YOLO processes frames directly
                all_tracks = []
                for frame_num, frame_img in frames:
                    tracks = tracker.update(None, frame_img)
                    all_tracks.append((frame_num, tracks))
                    
            elif tracker_name == 'BotSort':
                # BotSort uses pre-computed detections
                all_tracks = []
                for frame_num, frame_img in frames:
                    current_detections = detections_by_frame.get(frame_num, [])
                    detections_np = np.array(current_detections) if current_detections else np.array([])
                    
                    if len(detections_np) > 0:
                        tracks = tracker.update(detections_np, frame_img)
                    else:
                        tracks = np.array([])
                    
                    all_tracks.append((frame_num, tracks))
            
            tracking_results[tracker_name] = all_tracks
            
            end_time = time.time()
            processing_time = end_time - start_time
            fps = len(frames) / processing_time
            
            print(f"  Processed {len(frames)} frames in {processing_time:.2f}s ({fps:.1f} FPS)")
            
            # Get tracking statistics
            stats = tracker.get_statistics()
            print(f"  Total tracks: {stats['total_tracks']}")
            print(f"  Avg track length: {stats['avg_track_length']:.1f}")
    
    else:
        print(f"No detection file found for sequence: {test_sequence}")
        
else:
    print("No dataset available for processing.")
    print("This is a demonstration of the pipeline structure.")

## 6. Save Results

In [ ]:
# Save tracking results in MOT format
if 'tracking_results' in locals() and tracking_results:
    for tracker_name, all_tracks in tracking_results.items():
        output_file = Path(results_dir) / f"{test_sequence}_{tracker_name.lower()}.txt"
        
        print(f"Saving {tracker_name} results to: {output_file}")
        
        with open(output_file, 'w') as f:
            for frame_num, tracks in all_tracks:
                for track in tracks:
                    if len(track) >= 7:
                        x1, y1, x2, y2, track_id, conf, cls = track[:7]
                        
                        # Convert to MOT format
                        bb_left = x1
                        bb_top = y1
                        bb_width = x2 - x1
                        bb_height = y2 - y1
                        
                        # MOT format: frame,id,bb_left,bb_top,bb_width,bb_height,conf,x,y,z
                        line = f"{frame_num},{int(track_id)},{bb_left:.2f},{bb_top:.2f},{bb_width:.2f},{bb_height:.2f},{conf:.2f},-1,-1,-1\n"
                        f.write(line)
        
        print(f"  Saved {len([t for _, tracks in all_tracks for t in tracks])} track entries")

    print(f"\nAll results saved to: {results_dir}")
else:
    print("No tracking results to save.")

## 7. Generate Videos

In [ ]:
# Generate tracking videos for visualization
if 'tracking_results' in locals() and tracking_results and data_loader:
    video_output_dir = "../output/videos"
    ensure_dir(video_output_dir)
    
    for tracker_name, all_tracks in tracking_results.items():
        print(f"\nGenerating video for {tracker_name} tracker...")
        
        # Create annotated frames
        annotated_frames = []
        
        # Get frames and add tracking annotations
        frames = data_loader.load_sequence_frames(test_sequence)
        tracks_dict = {frame_num: tracks for frame_num, tracks in all_tracks}
        
        for frame_num, frame_img in frames:
            annotated_frame = frame_img.copy()
            
            # Draw tracks if available
            if frame_num in tracks_dict:
                tracks = tracks_dict[frame_num]
                if len(tracks) > 0:
                    # Use the tracker's draw method
                    if tracker_name in trackers:
                        annotated_frame = trackers[tracker_name].draw_tracks(annotated_frame, tracks)
            
            annotated_frames.append(annotated_frame)
        
        # Save video
        if annotated_frames:
            from src.utils.visualization import create_video_from_frames
            
            video_path = f"{video_output_dir}/{test_sequence}_{tracker_name.lower()}_tracking.mp4"
            success = create_video_from_frames(annotated_frames, video_path, fps=30)
            
            if success:
                print(f"  Video saved: {video_path}")
                
                # Display video in notebook
                if os.path.exists(video_path):
                    print(f"  Displaying {tracker_name} tracking video:")
                    display(Video(video_path, width=600))
            else:
                print(f"  Failed to create video for {tracker_name}")
                
else:
    print("No tracking results available for video generation.")

## 8. Results Analysis

In [ ]:
# Analyze and compare tracking results
if 'tracking_results' in locals() and tracking_results:
    print("=== Tracking Results Analysis ===")
    
    comparison_data = {}
    
    for tracker_name, all_tracks in tracking_results.items():
        # Count total tracks and detections
        all_track_ids = set()
        total_detections = 0
        frame_counts = []
        
        for frame_num, tracks in all_tracks:
            frame_counts.append(len(tracks))
            total_detections += len(tracks)
            
            for track in tracks:
                if len(track) >= 5:
                    track_id = int(track[4])
                    all_track_ids.add(track_id)
        
        comparison_data[tracker_name] = {
            'unique_tracks': len(all_track_ids),
            'total_detections': total_detections,
            'avg_detections_per_frame': np.mean(frame_counts) if frame_counts else 0,
            'max_detections_per_frame': max(frame_counts) if frame_counts else 0,
            'frames_processed': len(all_tracks)
        }
    
    # Display comparison table
    print("\nTracker Comparison:")
    print("-" * 70)
    print(f"{'Metric':<25} {'YOLO':<15} {'BotSort':<15} {'Difference':<15}")
    print("-" * 70)
    
    metrics = ['unique_tracks', 'total_detections', 'avg_detections_per_frame', 'max_detections_per_frame']
    
    for metric in metrics:
        yolo_val = comparison_data.get('YOLO', {}).get(metric, 0)
        botsort_val = comparison_data.get('BotSort', {}).get(metric, 0)
        diff = yolo_val - botsort_val if isinstance(yolo_val, (int, float)) else 'N/A'
        
        print(f"{metric.replace('_', ' ').title():<25} {yolo_val:<15.1f} {botsort_val:<15.1f} {diff:<15}")
    
    # Visualize comparison
    if len(comparison_data) >= 2:
        fig, axes = plt.subplots(2, 2, figsize=(12, 8))
        
        # Plot 1: Unique tracks
        trackers_list = list(comparison_data.keys())
        unique_tracks = [comparison_data[t]['unique_tracks'] for t in trackers_list]
        axes[0, 0].bar(trackers_list, unique_tracks)
        axes[0, 0].set_title('Unique Tracks Generated')
        axes[0, 0].set_ylabel('Number of Tracks')
        
        # Plot 2: Total detections
        total_dets = [comparison_data[t]['total_detections'] for t in trackers_list]
        axes[0, 1].bar(trackers_list, total_dets)
        axes[0, 1].set_title('Total Detections')
        axes[0, 1].set_ylabel('Number of Detections')
        
        # Plot 3: Average detections per frame
        avg_dets = [comparison_data[t]['avg_detections_per_frame'] for t in trackers_list]
        axes[1, 0].bar(trackers_list, avg_dets)
        axes[1, 0].set_title('Average Detections per Frame')
        axes[1, 0].set_ylabel('Detections per Frame')
        
        # Plot 4: Max detections per frame
        max_dets = [comparison_data[t]['max_detections_per_frame'] for t in trackers_list]
        axes[1, 1].bar(trackers_list, max_dets)
        axes[1, 1].set_title('Maximum Detections per Frame')
        axes[1, 1].set_ylabel('Max Detections')
        
        plt.tight_layout()
        plt.savefig('../output/plots/tracker_comparison.png', dpi=300, bbox_inches='tight')
        plt.show()
        
        print("Comparison plot saved to: ../output/plots/tracker_comparison.png")

else:
    print("No tracking results available for analysis.")

## 9. Command Line Usage Examples

In [ ]:
# Show how to use the command line scripts
print("=== Command Line Usage Examples ===")
print()
print("1. Run tracking with BotSort:")
print(f"   python scripts/run_tracking.py \\")
print(f"       --dataset {dataset_path} \\")
print(f"       --output {results_dir} \\")
print(f"       --tracker botsort \\")
print(f"       --device auto")
print()
print("2. Run tracking with YOLO:")
print(f"   python scripts/run_tracking.py \\")
print(f"       --dataset {dataset_path} \\")
print(f"       --output {results_dir} \\")
print(f"       --tracker yolo \\")
print(f"       --confidence 0.3")
print()
print("3. Evaluate results:")
print(f"   python scripts/evaluate_results.py \\")
print(f"       --gt_dir {dataset_path}/detections \\")
print(f"       --results_dir {results_dir} \\")
print(f"       --output evaluation_results.json")
print()
print("4. Download models:")
print("   python scripts/download_models.py --all")
print()
print("These scripts can be run from the command line or terminal.")

## Summary

This notebook demonstrated:

1. **Complete soccer tracking pipeline** for MOT datasets
2. **Multiple tracker comparison** (YOLO vs BotSort)
3. **Automated processing** of entire sequences
4. **Results visualization** and analysis
5. **Video generation** with tracking annotations
6. **Performance benchmarking** and statistics

### Key Features:
- ✅ MOT format dataset support
- ✅ Multiple tracking algorithms
- ✅ Automatic hardware detection (CPU/GPU)
- ✅ Results export in standard formats
- ✅ Comprehensive visualization tools
- ✅ Performance analysis and comparison

### Next Steps:
- Process your own soccer datasets
- Experiment with different tracking parameters
- Use the evaluation notebook for detailed metric analysis
- Scale up to process multiple sequences in batch

### For Production Use:
- Use the command-line scripts for batch processing
- Set up automated pipelines using the provided tools
- Monitor performance and optimize parameters
- Integrate with your existing workflow